# sample of Neural Network

In [1]:
import torch 
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self, input_size, hidden_size, out_size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, out_size)
    def forward(self, x):
        x = self.fc1(x)
        x=F.relu(x)
        x = self.fc2(x)
        return x

In [2]:
from sklearn import datasets

iris_dataset = datasets.load_iris()

In [3]:
iris_dataset.data.shape

(150, 4)

In [4]:
iris_dataset.target.shape

(150,)

In [5]:
iris_dataset.target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

There are three types of iris flower and data include four data: sepal length, sepal width, petal length, petal width

try to classify iris type from these data.

first, separate train and test data. Train data is used to fitting the model and test data is used to evaluate the model prediction

In [6]:
from sklearn.model_selection import train_test_split

train_X, test_X, train_y, test_y = train_test_split(iris_dataset.data, iris_dataset.target, test_size=0.2, stratify=iris_dataset.target, random_state=42)

In [7]:
train_X = torch.tensor(train_X, dtype=torch.float32)
test_X = torch.tensor(test_X, dtype=torch.float32)
train_y = torch.tensor(train_y, dtype=torch.int64)
test_y = torch.tensor(test_y, dtype=torch.int64)


First, very simple optimization scheme. Using each data one by one and optimize the parameters in network every steps

In [8]:
model=Net(4, 100, 3)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

for epoch in range(1000):
    out=model(train_X)
    loss=criterion(out, train_y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if epoch % 100 == 0:
        print('number of epoch', epoch, 'loss', loss.item())

number of epoch 0 loss 1.1787612438201904
number of epoch 100 loss 0.07345173507928848
number of epoch 200 loss 0.05522734299302101
number of epoch 300 loss 0.04852501302957535
number of epoch 400 loss 0.04449308291077614
number of epoch 500 loss 0.042108092457056046
number of epoch 600 loss 0.04093480482697487
number of epoch 700 loss 0.040312256664037704
number of epoch 800 loss 0.03995087370276451
number of epoch 900 loss 0.03966950625181198


In [9]:
predict_out=model(test_X)
_, predict_y = torch.max(predict_out, 1)

In [10]:
test_y, predict_y

(tensor([0, 2, 1, 1, 0, 1, 0, 0, 2, 1, 2, 2, 2, 1, 0, 0, 0, 1, 1, 2, 0, 2, 1, 2,
         2, 1, 1, 0, 2, 0]),
 tensor([0, 2, 1, 1, 0, 1, 0, 0, 2, 1, 2, 2, 2, 1, 0, 0, 0, 1, 1, 2, 0, 2, 1, 2,
         2, 1, 1, 0, 2, 0]))

In [11]:
n_test=len(test_X)
accuracy=(predict_y==test_y).sum().item()/n_test
print('accuracy is {} percent'.format(accuracy*100))

accuracy is 100.0 percent


Normally, to make the optimization more stable, we use mini-batch learning 

In [12]:
from torch.utils.data import DataLoader, TensorDataset

train_dataset = TensorDataset(train_X, train_y)
test_dataset = TensorDataset(test_X, test_y)

train_dataloader = DataLoader(train_dataset, batch_size=10, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=10, shuffle=False)

model=Net(4, 100, 3)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

for epoch in range(1000):
    for X, y in train_dataloader:
        out=model(X)
        loss=criterion(out, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if epoch % 100 == 0:
        print('number of epoch', epoch, 'loss', loss.item())

number of epoch 0 loss 0.5538369417190552
number of epoch 100 loss 0.0065150754526257515
number of epoch 200 loss 0.13144288957118988
number of epoch 300 loss 0.10834857076406479
number of epoch 400 loss 0.02163071371614933
number of epoch 500 loss 0.03838806226849556
number of epoch 600 loss 0.005226876121014357
number of epoch 700 loss 0.17277060449123383
number of epoch 800 loss 0.21886806190013885
number of epoch 900 loss 0.0022180473897606134


In [13]:
n_test=len(test_X)
accuracy=(predict_y==test_y).sum().item()/n_test
print('accuracy is {} percent'.format(accuracy*100))

accuracy is 100.0 percent
